In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)
dataset_path = 'gdrive/My Drive/Deep Learning/Insult classification/'

Mounted at /content/gdrive


In [ ]:
!unzip -q "gdrive/My Drive/Deep Learning/Insult classification/phase2.zip" -d "gdrive/My Drive/Deep Learning/Insult classification/phase2" 

In [ ]:
dataset_path = dataset_path + "phase2/phaseII/"

In [ ]:
import pandas as pd
import numpy as np
df_train = pd.read_csv(dataset_path+'train_text.csv')
df_test = pd.read_csv(dataset_path+'test_text.csv')

In [ ]:
df_train.head()

,id,Comment,Target
0,143508,I just did a geolocate. Having your friends d...,0
1,162092,"Thank you ). All the best, Tally-ho!",0
2,213301,"Here are a couple of recent thank yous OPs, fo...",0
3,143415,: \n That is an excellent choice and I am happ...,0
4,285402,Можешь говорить по русски.,0


In [ ]:
train_X, train_Y = df_train["Comment"].values, df_train["Target"].values

In [ ]:
df_test.head()

,id,Comment
0,163224,": Maybe they want, but they aren't. I have quo..."
1,248651,If you are curious about the issue it was disc...
2,210814,"::Haha, your desysop is in the mail! Yes, and ..."
3,118578,REDIRECT Talk:Communications and media in Sara...
4,228447,"::::::::I know you mean to help, but I wholly ..."


In [ ]:
test_X = df_test["Comment"].values

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

comment = []
lines = df_train["Comment"].values.tolist()

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
reviews = []
Y = train_Y
for line in lines:
  lst = line.split()
  lst = list(set(lst))
  line = ' '.join(lst)
  try:
    token = word_tokenize(line)
  except Exception as e:
    continue
  lower_token = [w.lower() for w in token]
  table = str.maketrans('','',string.punctuation)
  stripped = [w.translate(table) for w in lower_token]
  alpha_words = [w for w in stripped if w.isalpha()]
  #stop_words = set(stopwords.words('english'))
  #final_words = [w for w in alpha_words if w not in stop_words]
  #lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in final_words]
  reviews.append(alpha_words)

In [ ]:
test_reviews = []
test_lines = df_test["Comment"].values.tolist()
#Y_t = test_Y
for i,line in enumerate(test_lines):
  lst = line.split()
  lst = list(set(lst))
  line = ' '.join(lst)
  try:
    token = word_tokenize(line)
  except Exception as e:
    #Y_t = np.delete(Y_t,i)
    continue
  lower_token = [w.lower() for w in token]
  table = str.maketrans('','',string.punctuation)
  stripped = [w.translate(table) for w in lower_token]
  alpha_words = [w for w in stripped if w.isalpha()]
  '''stop_words = set(stopwords.words('english'))
  final_words = [w for w in alpha_words if w not in stop_words]
  lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in final_words]'''
  #lst = list(set(lemmatized_words))
  test_reviews.append(alpha_words)

In [ ]:
embeddings_index = dict()
f = open('gdrive/My Drive/Deep Learning/Insult classification/glove.840B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    try:
      coefs = np.asarray(values[1:],dtype='float32')
      embeddings_index[word] = coefs
    except:
      continue
f.close()


In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
reviews[0]

'geolocate meatpuppet something friend'

In [ ]:
final_data = [' '.join(x) for x in reviews] + [' '.join(x) for x in test_reviews]

In [ ]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(test_reviews+reviews)
sequences = tokenizer_obj.texts_to_sequences(reviews)
word_index = tokenizer_obj.word_index
max_length= max([len(str(s).split()) for s in final_data])
review_pad = pad_sequences(sequences,maxlen= max_length)

In [ ]:
print(max_length)
num_words = len(word_index)+1

583


In [ ]:
num_words = len(word_index)+1
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))
for word,i in word_index.items():
  if i> num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import *
from keras.layers import *
from keras.initializers import Constant

TIME_STEPS = max_length
INPUT_DIM = max_length

def attention_3d_block(inputs,TIME_STEPS,n):
    # inputs.shape = (batch_size, time_steps, input_dim)
    
    
    a = Permute((2, 1))(inputs)
    
    a = Dense(int(TIME_STEPS), activation='softmax')(a)
    
    
    a_probs = Permute((2, 1), name='vec'+str(n))(a)
  
    output_attention_mul = Multiply()([inputs, a_probs])
    #output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    n+=1
    return output_attention_mul


def model_attention_applied_after_lstm():
    inputs = Input(shape= (INPUT_DIM,))
    lstm_units = 64
    embedding_layer = Embedding(num_words,
                                300,
                                embeddings_initializer = Constant(embedding_matrix),
                                input_length = max_length,
                                trainable=False)
    inputs_1 = embedding_layer(inputs)
    c1 = Conv1D(64,kernel_size=3,padding='same',activation='relu')(inputs_1)
    c2 = Conv1D(64,kernel_size=3,padding='same',activation='relu')(c1)
    p1 = MaxPool1D(pool_size=(2))(c2)
    d1 = Dropout(0.5)(p1)
    TIME_STEPS = d1.shape[1]
    #print(TIME_STEPS)
    attention_mul = attention_3d_block(d1,TIME_STEPS,0)
    #attention_mul = Flatten()(attention_mul)
    
    g1 = GRU(128, dropout =0.2, recurrent_dropout = 0.2, return_sequences=True)(attention_mul)
    #g2 = GRU(128, dropout =0.2, recurrent_dropout = 0.2, return_sequences=True)(g1)
    
    attention_mul = attention_3d_block(g1,TIME_STEPS,1)
    attention_mul = Flatten()(attention_mul)
    
    output_1 = Dense(128, activation='relu')(attention_mul)
    output = Dense(1, activation='sigmoid')(output_1)

    
    model = Model(input=[inputs], output=output)
    return model

In [ ]:
from keras.optimizers import Adam
m = model_attention_applied_after_lstm()

m.compile(optimizer=Adam(lr = 0.001), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [ ]:
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           (None, 568)          0                                            
__________________________________________________________________________________________________
embedding_33 (Embedding)        (None, 568, 300)     71275500    input_34[0][0]                   
__________________________________________________________________________________________________
conv1d_65 (Conv1D)              (None, 568, 64)      57664       embedding_33[0][0]               
__________________________________________________________________________________________________
conv1d_66 (Conv1D)              (None, 568, 64)      12352       conv1d_65[0][0]                  
__________________________________________________________________________________________________
max_poolin

A secondary model:

This is a single model consisting of a combination of Conv1D layers and a LSTM/GRU layer, followed by a dense layer

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Embedding, CuDNNGRU, SimpleRNN, LSTM, Bidirectional,Conv1D,MaxPooling1D,Dropout,Flatten,MaxPool1D
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.optimizers import Adam

model = Sequential()
embedding_layer = Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix),
                            input_length = max_length, trainable = False)
model.add(embedding_layer)
model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model.add(MaxPool1D(pool_size=(2)))
model.add(Dropout(0.5))
model.add(CuDNNGRU(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer=Adam(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
print(embedding_matrix.shape)

(237586, 300)


In [ ]:
Y_ = np.asarray(Y).reshape(len(Y),1)

In [ ]:
from keras.callbacks import ModelCheckpoint
filepath= "gdrive/My Drive/Deep Learning/Insult classification/weights-improvement-{epoch:02d}-{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]
model.fit(review_pad,Y_,validation_split = 0.2,epochs = 10,batch_size = 128,verbose =1,callbacks = callbacks_list)

Train on 81260 samples, validate on 20315 samples
Epoch 1/10
81260/81260 [==============================] - 66s 818us/step - loss: 0.5524 - acc: 0.7458 - val_loss: 0.5222 - val_acc: 0.7642

Epoch 00001: val_loss improved from inf to 0.52216, saving model to gdrive/My Drive/Deep Learning/Insult classification/weights-improvement-01-0.76.h5
Epoch 2/10
81260/81260 [==============================] - 64s 786us/step - loss: 0.5278 - acc: 0.7612 - val_loss: 0.5233 - val_acc: 0.7649

Epoch 00002: val_loss did not improve from 0.52216
Epoch 3/10
81260/81260 [==============================] - 64s 785us/step - loss: 0.5194 - acc: 0.7651 - val_loss: 0.5222 - val_acc: 0.7633

Epoch 00003: val_loss did not improve from 0.52216
Epoch 4/10
81260/81260 [==============================] - 64s 785us/step - loss: 0.5107 - acc: 0.7684 - val_loss: 0.5250 - val_acc: 0.7632

Epoch 00004: val_loss did not improve from 0.52216
Epoch 5/10
81260/81260 [==============================] - 64s 785us/step - loss: 0.500

KeyboardInterrupt: ignored

In [ ]:
model.save('gdrive/My Drive/Deep Learning/Insult classification/weights-improvement-epoch:02-val_acc:.76.h5')

In [ ]:
from keras.models import load_model
model_best = load_model('gdrive/My Drive/Deep Learning/Insult classification/weights-improvement-04-0.76.h5')

In [ ]:
sequences = tokenizer_obj.texts_to_sequences(test_reviews)
review_pad = pad_sequences(sequences,maxlen= max_length)

In [ ]:
'''test_id = df_test["id"].values
output = model_best.predict(review_pad, verbose=1, batch_size=128)
output = [l[0] for l in output]'''
for i in range(len(output)):
  if output[i]<0.10:
    output[i]=0.10
  elif output[i]>0.9:
    output[i] = 0.9
out_df = pd.DataFrame(list(zip(test_id, output)), columns =['id', 'Prediction'])
out_df.to_csv("gdrive/My Drive/Deep Learning/Insult classification/output.csv")

In [ ]:
output[0]

0.2975562

In [ ]:
len(test_id)

73556

In [ ]:
df_test.loc[0]

id                                                    163224
Comment    : Maybe they want, but they aren't. I have quo...
Name: 0, dtype: object